In [ ]:
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['pdf.fonttype'] = 42

## Figure 2a

In [ ]:
from src.CotrxMatrix import CotrxMatrix, combine_reps
from src.DMS_Profile import Genome

In [ ]:
# load rRNA sequence
genome_rrna = Genome('/Users/leo/Documents/repos/lsStruct/data/genome/scer_chr_kan.FASTA', coords=(11, 451575, 458433), reverse=True)

# dot bracket secondary structure
dbr_18 = '...((((.........))))((((.(((((((.(((((((((.....(((.(((..((...(((..(.((..........)))..))))).....((((.......(((((((..((..(((((((............(((((....(((((((.....)))))))....)))))......(((((((((....)))))))))(((.(((((((.......(((((.(((....)))...))))).....))))))).)..))...((((.((((.....))))))))..))))))).))))))))).(((..(.(((....((((((((.......))))))))))).....))))...((((((((....))))...))))))))((((((..........)))))).((((....))))...)))))))......(.(((...(((((...))))).)))).)).))))))....((((..(((((((....)))))))..).))).....((((((((.......))))))))........((.((......(.((((((..(((....)))....))))))))).)).))))))))))).....(...(((.......((((...(((.((....((((((((((((.((((.(((.....)))...)))).....))))))))))))....((((((....(((((((((.....)))))))))....))))))..(((((((((.......(((..(.(...).)..(((.....)))...)))......)))))..)))).....(.((....(.((.(((.............))).))..)..)).)..))...((((((((((.((((((((((((((((((((...(((......)))......))))))))))))....(..((....)))))))))))))))).))))..))))...)))).(..((((((...(((.(((((.........))))).)))))))))..).......((((((.(((..(((((((...((..........)))))))))..)))...((....))...)))....))).))))(((((.((.((((....)))))))))))........(((((.(((((((..((..(((((((((((((((((.(.)((((........))))........(((((((....(((((....(((((((((..........)))))))))..))))).(.((.((((..((((((((((..(((((((((....)))..((((......))))..)))))).....((((((((.((((..(((((.((((((....))))))...)))))..))))))).((.(((((((...)))))))))....)))))...))))).)))...).))))))))....)))))))...)).)))))))))((..(((((((.(...(((.....(((.((((....)))).)))....)))....).)))))))....).((((((((((((........))))))))))))..).))))))(...(((((((((.......)))))))))..)..))...)))))))))).))....((.((...(((((((((((.((((((((((((..(((((((((((((((((((((((((((....)))))))))))))))))))))))))))..)))))))))))))))))))))))....))..))....((((((((((....))))))))))........'
dbr_58 = '........................................((((((((......)))).....((((((....................)))).))...))))...(((...)))(((((((((....))))))))).....................'
dbr_25 = '..........................((((((.....((....))......))))).).........((....))..(((((......((.....)).....))))).((((..((((((((((((((((((...))))))))))))))))))....(((((((.((((((((((((((((((((...(.(((((....))))))..(...).(((.......)))....))))))))..))))))))....)))))))))))........(((((((........))))))).....(((((........)))))..)))).................................((....))...............((((....))))..................................((((((((((((.(..(((((...((((..((((((((.....))))))))......))))...))))).(((((((((((((((((((((((......((((((((..((((((((((....))))))..)))).))))))))....))))).))))))))))).......(((....))).....)))))))..).))))))))))))..((((........))))((((((....(((((((((((((..((((..((((.....))))..))))...((....)).......(((((..(((((....)))))...))))).((((((((...((....))...))).))).))....)))))))))))))..((((...((((((...((((((((((((.((((...(((((((....)))))))....))))...((((.((.((........))))))))..).))))))))))).(((....))).......))))))......))))..((((((....(((((....))))).(((((((((.......((((((((..(.(((....((((((((((..((((((((.....))))))))))))))))))..)).).)..)))))))).(((((((((.........)))))))))...........)))))))).)...((((((((((.....))))))))))....(((......)))...(((((((...(((......(.((((((.................(((.((((((((((.......(((((.(((..(((.........)))..)))(...)(...((......)).).)))))...)))))))))))))...................))))))))))...)))))))(((((((((.........)))))))))....))))))(.((((((((.((......((((.((((....))))))))...))))))))))..).....))))))((((.((((.....((((((.......))))))((((.....(((((.....(((((((...((.....)).))))))).....(((.((...(((((.(((....))))))......(((((........))))).....))....))...(((((........)))))..((((.(((((((.....(((....))).......((((((...((((.(((((((.(((((((((.........))))))...(((((((..(((((((((((((.....)))))))))))))..)).)))))......(((((((....)))))))..)))...))))))).)))).........))))))............))))))))))).))).........((......))))))).((((.....))))..))))..((((....((((((((((((..............(((((...((....))....)))))((((((((((((((.((((((((((((((((((....(((((((((((((((((((.(((((((((((((....))))))))))))).))))))))))))).))))))....))))))))))(((((.....)))))....))))))))......)))))..)))))))))(...).......(((((((((..(((.........)))..(((((((((..((((.....))))..)..))))))))..((((.(((..((((((.(..(((....(((((....)))))....))))))))..)))))((((((.......))))))(...).((((.(...(.)).(((((((.)..))))))..)..)))..))))...)).)))))))....)))))))))))).))))))))))))....(((((........)))))..((((((((.(((.((.....(((((.(((((((.(((((..(((((((..((((((((((((((((((.........(((((..((....))............))))))))))))).))))))))))...(((((.....((((..(((((......)))))...............((((((....))))))..))))..))))).....)))))))...)))))..((((.....))).)((((((...........))))))..((..((((((((((.((((((.......))))))...))((......))....))))))))..(((((((((.......))))))))).(((....))).))..........((((((..((((.......)))))))))).........)))))..))))))).......(((..(((((((...((.......))...)))))))...)))..........(((((((((((..((((((((((....))))))))..)).(((((.....))))).....)))))..)..)))).).....(((((((....))).))))....))..)))))))))))..((((((((..((((.((((((((...............))))))))(((((...(((....(((((((((((....))))))..)))))...)))..)))))..(((....(((((..........)))))....))).))))....)))))))).......(((((((((((((................((((((.....))))))...............((((((((..(((((((((((......))))))))))))))))))).............)))))))))))))..((((((((((.......)))))....(((((((((.((..(..((((.((((((((......)))))))))))))...))....))).))))))..)))))..'
dbr = 701*'?' + dbr_18 + 361*'?' + dbr_58 + 232*'?' + dbr_25 + 210*'?'

# load co-transcriptional folding data
min_cov = 900

wild_1 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_1_rRNA.pkl', 'kk', genome_rrna.seq, '-')
wild_2 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_2_rRNA.pkl', 'kk', genome_rrna.seq, '-')
wild_3 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_3_rRNA.pkl', 'kk', genome_rrna.seq, '-')
wild_4 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_4_rRNA.pkl', 'kk', genome_rrna.seq, '-')
wild_5 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_5_rRNA.pkl', 'kk', genome_rrna.seq, '-')
wild_6 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_6_rRNA.pkl', 'kk', genome_rrna.seq, '-')
wild_7 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_7_rRNA.pkl', 'kk', genome_rrna.seq, '-')
wild_8 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_8_rRNA.pkl', 'kk', genome_rrna.seq, '-')

all_dms = combine_reps([wild_1, wild_2, wild_3, wild_4, wild_5, wild_6, wild_7, wild_8], 'all')
all_dms.process(min_cov=min_cov)

In [ ]:
coords = (380, 680)

# save matrix as image
all_dms.save_to_img('matrix_image.png', coords=coords, filter_ac=False, cmap_loc='../cmap.txt')

# plot profiles for individual polymerase positions
rea_dict = all_dms.plot_single_pol_pos([456, 468, 472], cmap_loc='../cmap.txt')

## Figure 2b

In [ ]:
from src.dms_utils import plot_structure, get_coords

In [ ]:
# updated structure of 5'ETS based on CoSTseq data
ets_mby = '.((((((((((..(((((((((((.......(((((((....)))))))......))))))).))))...)))))))))).........((((((((.((...((((..(((((((((((....)))).)))))))))))...)).)))......)))))..(((((((.(((((..((((((((((.((((((((....)))))))).)))))))))).)))))))))))).......(((((((((((...........)))).))))))).....(((...(((((((((((.(((.....))).))))))..)))))...))).....(((((..(((((((..((((...(((((....)))))..))))..)))).)).)))))).....((((((...(((((((((((((...((((.....).))))))).).))))))))....)))))).|||||...||||||||||..............((((((((.(((.((((((((......)))).)))).)))...))).)))))(((((.((((..((.((((((....)))))).))..)))).)))))........................((.((((((.(((((((((.((((((..((((((((....)))))))).)))))).)))))).)))))))))))...........'
ets_seq = genome_rrna.seq.replace('T', 'U')[1:701]
ets_coo_mby = get_coords(ets_seq, ets_mby)

In [ ]:
# plot co-transcriptional DMS reactivities onto ETS structure
fig, axs = plt.subplots(3, 1, figsize=(8, 20))
fig.patch.set_facecolor('white')

(x1, x2) = (390, 474)
plot_structure(ets_coo_mby[x1:x2], ets_seq[x1:x2], rea_dict[456][1][x1+1:x2+1], ~(genome_rrna.mask_G()|genome_rrna.mask_U())[x1+1:x2+1], axs=axs[0], circle_size=100, text_size=6, cmap_loc='../cmap.txt')
axs[0].set_title('Pol I position 456')
plot_structure(ets_coo_mby[x1:x2], ets_seq[x1:x2], rea_dict[468][1][x1+1:x2+1], ~(genome_rrna.mask_G()|genome_rrna.mask_U())[x1+1:x2+1], axs=axs[1], circle_size=100, text_size=6, cmap_loc='../cmap.txt')
axs[1].set_title('Pol I position 468')
plot_structure(ets_coo_mby[x1:x2], ets_seq[x1:x2], rea_dict[472][1][x1+1:x2+1], ~(genome_rrna.mask_G()|genome_rrna.mask_U())[x1+1:x2+1], axs=axs[2], circle_size=100, text_size=6, cmap_loc='../cmap.txt')
axs[2].set_title('Pol I position 472')

plt.show()

## Figure S2a and S2b

In [ ]:
from src.DMS_Profile import Targeted_DMS_Profile, targeted_combine_profiles
from src.dms_utils import rsample
import pickle

In [ ]:
# load nascent and mature average rRNA data
genome_rrna = Genome('/Users/leo/Documents/repos/lsStruct/data/genome/scer_chr_kan.FASTA', coords=(11, 451575, 458433), reverse=True)

wild_nas_1 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_1_rRNA_agg.pkl', 'wild_R1', genome_rrna, reverse=True) 
wild_nas_2 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_2_rRNA_agg.pkl', 'wild_R2', genome_rrna, reverse=True)
wild_nas_3 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_3_rRNA_agg.pkl', 'wild_R3', genome_rrna, reverse=True)
wild_nas_4 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_4_rRNA_agg.pkl', 'wild_R3', genome_rrna, reverse=True)
wild_nas_5 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_5_rRNA_agg.pkl', 'wild_R3', genome_rrna, reverse=True)
wild_nas_6 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_6_rRNA_agg.pkl', 'wild_R3', genome_rrna, reverse=True)
wild_nas_7 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_7_rRNA_agg.pkl', 'wild_R3', genome_rrna, reverse=True)
wild_nas_8 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_8_rRNA_agg.pkl', 'wild_R3', genome_rrna, reverse=True)
wild_nas = targeted_combine_profiles([wild_nas_1, wild_nas_2, wild_nas_3, wild_nas_4, wild_nas_5, wild_nas_6, wild_nas_7, wild_nas_8], 'wild', genome_rrna, per_exclude=10)

In [ ]:
# known 5'ETS structure
ets_dbr_kno = '.((((((((((..(((((((((((.......(((((((....)))))))......))))))).))))...)))))))))).........((((((((.((...((((..(((((((((((....)))).)))))))))))...)).)))......)))))..(((((((.(((((..((((((((((.((((((((....)))))))).)))))))))).)))))))))))).......(((((((((((...........)))).))))))).......|||||||||||.......(((...))).........................(((((..(((((((..((((...(((((....)))))..))))..)))).)).)))))).....((((((...(((((((((((((...((((.....).))))))).).))))))))....)))))).|||||...||||||||||..............((((((((.(((.((((((((......)))).)))).)))...))).)))))(((((.((((..((.((((((....)))))).))..)))).)))))........................((.((((((.(((((((((.((((((..((((((((....)))))))).)))))).)))))).)))))))))))...........'

# structure prediction for the region of disagreement with known structure (nucleotides 274:331)
_, E, ets_dbr_pre = rsample(genome_rrna.seq[274:331], (wild_nas.rat*wild_nas.vec_norm)[274:331], (wild_nas.mask & ~genome_rrna.mask_G())[274:331], rnastructure_path='/Users/leo/Builds/RNAstructure/exe', md=9999, ns=10000, t=303.15)

# calculate coordinates for both 5'ETS variants
coords_kno = get_coords(genome_rrna.seq[1:701], ets_dbr_kno)
coords_pre = get_coords(genome_rrna.seq[1:701], ets_dbr_kno[:274] + ets_dbr_pre + ets_dbr_kno[331:])

# draw figure
fig, axs = plt.subplots(2, 1, figsize=(20, 8))
fig.patch.set_facecolor('white')

plot_structure(coords_kno, genome_rrna.seq[1:701], (wild_nas.rat*wild_nas.vec_norm)[1:701], (wild_nas.mask & ~genome_rrna.mask_G())[1:701], axs=axs[0], circle_size=30, text_size=5, cmap_loc='../cmap.txt')
axs[0].set_title("5'-ETS post U3 annealing")
plot_structure(coords_pre, genome_rrna.seq[1:701], (wild_nas.rat*wild_nas.vec_norm)[1:701], (wild_nas.mask & ~genome_rrna.mask_G())[1:701], axs=axs[1], circle_size=30, text_size=5, cmap_loc='../cmap.txt')
axs[1].set_title("5'-ETS according to CoSTseq data")

plt.show()

## Figure S2c

In [ ]:
from src.pydscatter import dscatter_plot, dscatter_contour
from sklearn.cluster import KMeans
from tqdm import tqdm
import matplotlib.colors as mplc

def find_transitions_to_paired(mut, min_traj_len=10, n_final_state=3, n_unpaired_state=2):

    # append all trajectories together to estimate states
    all_x = np.array([0])
    all_y = np.array([0])
    all_i = []
    for res in range(mut.size[0]):
        if mut.mask_seq[res]:

            x, y = mut.get_residue_txn(res, min_cov=800, binned=False)

            if x.shape:
                x = x[y>0]
                y = np.log10(y[y>0])

                if len(y) >= min_traj_len:
                    all_x = np.hstack([all_x, x-x[0]+1+all_x[-1]])
                    all_y = np.hstack([all_y, y])
                    all_i += [res]*len(x)

    all_y = all_y[1:]
    all_y = (all_y - np.min(all_y)) / (np.max(all_y) - np.min(all_y))

    means, stds = state_finding_kmeans(all_y, 4)
    states = assign_states(all_y, means, stds)
    y_hat = [means[i] for i in states]

    # now let's find trajectories that end in unpaired state
    state_sort = np.argsort(means)
    
    become_paired = []
    paired_when = []
    for res in np.unique(all_i):  
        state_res = states[all_i==res]

        if all(state_res[-n_final_state:] == state_sort[0]) and np.sum((state_res == state_sort[-1]) | (state_res == state_sort[-2])) >= n_unpaired_state:
            i = 0
            while state_res[::-1][i] == state_sort[0]:
                i += 1
            paired_when.append(len(state_res)-(i+1))
            become_paired.append(res)
            
    return(means, stds, states, y_hat, become_paired, paired_when)


def assign_states(data, means, stds):
    # Assign each data point to one of the states based on the closest mean value.

    n_states = len(means)
    states = np.zeros(len(data), dtype=int)
    for i, d in enumerate(data):
        dists = np.abs(d - means) / stds
        states[i] = np.argmin(dists)
    return(states)


def state_finding_kmeans(y, n_states):

    # Normalize the data
    data_norm = (y - np.mean(y)) / np.std(y)

    # Apply K-means clustering
    kmeans = KMeans(n_clusters=n_states, random_state=0, n_init='auto').fit(data_norm.reshape(-1, 1))

    # Assign each data point to a state based on the closest mean value
    means = kmeans.cluster_centers_ * np.std(y, axis=0) + np.mean(y, axis=0)
    means = [i[0] for i in means]
    stds = []
    for i in range(n_states):
        cluster_data = y[kmeans.labels_ == i]
        stds.append(np.std(cluster_data))
    
    return(means, stds)

In [ ]:
# load Co-transcriptional matrices for wildtype and mutants that don't affect Pol I

wild_1 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_1_rRNA.pkl', 'wild_R1', genome_rrna.seq, '-')
wild_2 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_2_rRNA.pkl', 'wild_R2', genome_rrna.seq, '-')
wild_3 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_3_rRNA.pkl', 'wild_R3', genome_rrna.seq, '-')
wild_4 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_4_rRNA.pkl', 'wild_R4', genome_rrna.seq, '-')
wild_5 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_5_rRNA.pkl', 'wild_R5', genome_rrna.seq, '-')
wild_6 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_6_rRNA.pkl', 'wild_R6', genome_rrna.seq, '-')
wild_7 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_7_rRNA.pkl', 'wild_R7', genome_rrna.seq, '-')
wild_8 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_8_rRNA.pkl', 'wild_R8', genome_rrna.seq, '-')
wild = combine_reps([wild_1, wild_2, wild_3, wild_4, wild_5, wild_6, wild_7, wild_8], 'wild')
wild.process()

spt4_1 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_spt4_1_rRNA.pkl', 'spt4_R1', genome_rrna.seq, '-')
spt4_2 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_spt4_2_rRNA.pkl', 'spt4_R2', genome_rrna.seq, '-')
spt4_3 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_spt4_3_rRNA.pkl', 'spt4_R3', genome_rrna.seq, '-')
spt4 = combine_reps([spt4_1, spt4_2, spt4_3], 'spt4')
spt4.process()

dbp3_1 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_dbp3_1_rRNA.pkl', 'dbp3_R1', genome_rrna.seq, '-')
dbp3_2 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_dbp3_2_rRNA.pkl', 'dbp3_R2', genome_rrna.seq, '-')
dbp3_3 = CotrxMatrix('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_dbp3_3_rRNA.pkl', 'dbp3_R3', genome_rrna.seq, '-')
dbp3 = combine_reps([dbp3_1, dbp3_2, dbp3_3], 'dbp3')
dbp3.process()

In [ ]:
# find pairing status using kmeans, extract nucleotides that don't pair immediately
paired_whens = []
become_paireds = []
xs = []
ys = []
for i, mut in enumerate(tqdm([wild, spt4, dbp3])):
    
    means, stds, states, y_hat, become_paired, paired_when = find_transitions_to_paired(mut)
    paired_whens.append(paired_when)
    become_paireds.append(become_paired)
    for res in become_paired:
        x, y = mut.get_residue_txn(res, min_cov=800, binned=False)
        x = x-x[0]
        xs.append(x)
        ys.append(y)

In [ ]:
# draw figure
plt.figure(figsize=(5, 3))

cmap = np.array(pd.read_csv('../cmap.txt', header=None))/255
cmap = mplc.ListedColormap(cmap)

dscatter_plot(np.hstack(xs), np.hstack(ys), nbins=(200, 800), rasterized=True, cmap=cmap)
dscatter_contour(np.hstack(xs), np.hstack(ys), nbins=(200, 800), cmap='Greys')
plt.ylim(0, 1)
plt.xlabel('Nucleotides synthesized')
plt.ylabel('DMS reactivity')
plt.show()